In [1]:
from __future__ import division
from __future__ import print_function
import tensorflow as tf
import numpy as np
import scipy.sparse as sp
import pickle
from decagon.deep.minibatch import EdgeMinibatchIterator

In [2]:
noise = 0.05

In [30]:
mb_file = 'data/data_structures/MINIBATCH/MINIBATCH_toy_genes_500_drugs_400_se_4_batchsize_512_valsize_0.15'
with open(mb_file, 'rb') as f:
    minibatch = pickle.load(f)

In [31]:
for edge in minibatch.test_edges:
    #for p in minibatch.test_edges[edge]:
    print(np.shape(minibatch.test_edges[edge]))

(1, 53, 2)
(1, 53, 2)
(1, 92, 2)
(4,)


In [32]:
for edge in minibatch.test_edges:
    for p in range(np.shape(minibatch.test_edges[edge])[0]):
        print('Edge:',edge,p)
        data_true = minibatch.test_edges[edge][p]
        data_false = minibatch.test_edges_false[edge][p]
        assert len(data_true)==len(data_false), 'Dimension of true and false mismatch'
        l = len(data_true)
        print('Number of edges:',l)
        #data = np.concatenate((data_true.reshape(l,2,1),data_false.reshape(l,2,1)),axis=2)
        full_idx = np.arange(2*l)
        n_idx = np.floor(noise*2*l).astype(np.int)
        idx = np.random.choice(full_idx,size=n_idx)
        col = np.zeros(len(idx))
        print('Number of noise edges:',len(idx))
        # We choose a convention that indices from l to 2l are from the true dataset
        for i in range(len(idx)):
            if idx[i] >= l:
                idx[i] = idx[i]-l
                col[i] = 1
        col = col.astype(np.bool)
        false_to_true = idx[~col]
        print('Number of items changed from false to true',len(false_to_true))
        true_to_false = idx[col]
        print('Number of items changed from true to false',len(true_to_false))
        data_true = np.concatenate((data_true,data_false[false_to_true]),axis=0)
        print('Temporal extended length',len(data_true))
        data_false = np.concatenate((data_false,data_true[true_to_false]),axis=0)
        print('Temporal extended length',len(data_false))
        minibatch.test_edges[edge][p] = np.delete(data_true, true_to_false, axis=0)
        print('Final length',len(minibatch.test_edges[edge][p]))
        minibatch.test_edges_false[edge][p] = np.delete(data_false, false_to_true, axis=0)
        print('Final length',len(minibatch.test_edges_false[edge][p]))

Edge: (0, 1) 0
Number of edges: 53
Number of noise edges: 5
Number of items changed from false to true 2
Number of items changed from true to false 3
Temporal extended length 55
Temporal extended length 56
Final length 52
Final length 54
Edge: (1, 0) 0
Number of edges: 53
Number of noise edges: 5
Number of items changed from false to true 2
Number of items changed from true to false 3
Temporal extended length 55
Temporal extended length 56
Final length 52
Final length 54
Edge: (0, 0) 0
Number of edges: 92
Number of noise edges: 9
Number of items changed from false to true 1
Number of items changed from true to false 8
Temporal extended length 93
Temporal extended length 100
Final length 85
Final length 99
Edge: (1, 1) 0
Number of edges: 3112
Number of noise edges: 311
Number of items changed from false to true 161
Number of items changed from true to false 150
Temporal extended length 3273
Temporal extended length 3262
Final length 3125
Final length 3104
Edge: (1, 1) 1
Number of edges: